In [1]:
import dill
import pathlib
import pandas as pd
import numpy as np
from collections import defaultdict

from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from window_based_tagger_config import get_config
from FindFiles import find_files
from DirUtils import dir_exists
from results_procesor import ResultsProcessor, __MICRO_F1__
from Settings import Settings

In [2]:
EMPTY_TAG = "Empty"
ANAPHORA = "Anaphor"

In [3]:
!python -V

Python 3.6.4 :: Anaconda, Inc.


In [4]:
!which python

/Users/simon.hughes/anaconda3/envs/phd_py36/bin/python


In [5]:
!pip freeze

absl-py==0.3.0
anaconda-client==1.6.11
appnope==0.1.0
argcomplete==1.9.4
asn1crypto==0.24.0
astor==0.7.1
beautifulsoup4==4.6.0
bleach==2.1.2
boto==2.47.0
boto3==1.5.36
botocore==1.8.50
bz2file==0.98
certifi==2018.1.18
cffi==1.11.4
chardet==3.0.4
clyent==1.2.2
costcla==0.5
cryptography==2.1.4
cycler==0.10.0
cymem==1.31.2
cytoolz==0.8.2
decorator==4.2.1
dicecore==1.13
dill==0.2.8.2
docutils==0.14
entrypoints==0.2.3
ftfy==4.4.3
gast==0.2.0
gensim==0.13.4
grpcio==1.14.0
h5py==2.7.0
hdbscan==0.8.12
html5lib==1.0.1
idna==2.6
ipykernel==4.8.2
ipython==6.2.1
ipython-genutils==0.2.0
ipywidgets==7.1.2
jedi==0.11.1
Jinja2==2.10
jmespath==0.9.3
joblib==0.9.4
jsonschema==2.6.0
jupyter==1.0.0
jupyter-client==5.2.2
jupyter-console==5.2.0
jupyter-core==4.4.0
jupyterlab==0.31.8
jupyterlab-launcher==0.10.5
Keras==1.2.2
Keras-Applications==1.0.4
Keras-Preprocessing==1.0.2
Markdown==2.6.11
MarkupSafe==1.0
matplotlib==2.0.0
mistune==0.8.3
murmurhash==0.26.4
nb-anacondacloud==1.4.0
nb-conda==2.2.1
nb-conda-

In [7]:
# DATASET = "CoralBleaching"
DATASET = "SkinCancer"

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
training_pickled = settings.data_directory + DATASET + "/Thesis_Dataset/training.pl"

# PREDICTIONS FOLDERS
anaphor_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/"

# USE the "-Fixed" ones, these are much worse
#anaphor_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-Anaphora_Tags-Binary/"

config = get_config(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [8]:
anaphor_predictions_folder

'/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/'

In [9]:
assert dir_exists(anaphor_predictions_folder)
print("Valid")

Valid


In [10]:
# get tagged essays
def load_tagged_essays(folder, pattern):
    files = find_files(folder, pattern)
    # multiple runs with different hidden layer sizes?
    if len(files) > 2:        
        for f in files:
            print(f)
    assert len(files) == 2, "Wrong number of tagged files:" + str(len(files))
    for f in files:
        assert "_train_" in f or "_test_" in f, "Wrong files matched"
    train_tagged_fname = [f for file in files if "_train_" in f][0]
    test_tagged_fname = list(set(files).difference([train_tagged_fname]))[0]
    print("Train: {fname}".format(fname=train_tagged_fname))
    print("Test:  {fname}".format(fname=test_tagged_fname))

    # NOTE - is this throws an error, upgrade to dill 2.8.2. Version 2.6 had a bug in it
    with open(train_tagged_fname, "rb") as f:
        tagged_essays_train = dill.load(f)
    with open(test_tagged_fname, "rb") as f:
        tagged_essays_test  = dill.load(f)
    return (tagged_essays_train, tagged_essays_test)

In [11]:
# for optimal vd params - see http://localhost:8888/notebooks/Mongo%20Queries/Query%20Model%20Hyper%20Parameter%20Tuning%20Results-Anaphora%20Tagger.ipynb#
# NOTE that 1 layer is optimal for SC, 2 for CB
pattern = "essays_.*_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-1_use_pretrained_embedding-True.dill"
ana_tagged_tr, ana_tagged_test = load_tagged_essays(anaphor_predictions_folder, pattern=pattern)
len(ana_tagged_tr), len(ana_tagged_test)

Train: /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-1_use_pretrained_embedding-True.dill
Test:  /Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/essays_test_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-1_use_pretrained_embedding-True.dill


(870, 218)

In [12]:
# Train: /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/Predictions/
#     Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill

# Test:  /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/Predictions/
#     Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/essays_test_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill

## Load Raw Essays (Untagged)

In [13]:
# do I need to do this? good for validation below, but not needed otherwise
# with open(training_pickled, "rb+") as f:
#     untagged_essays_train = pickle.load(f)
untagged_essays_train = load_process_essays(**config)

test_config = get_config(test_folder)
untagged_essays_test = load_process_essays(**test_config)

len(untagged_essays_train), len(untagged_essays_test) # 870, 218

870 files found
870 essays processed
218 files found
218 essays processed


(870, 218)

In [14]:
config

{'folder': '/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Training/',
 'include_normal': False,
 'include_vague': True,
 'lower_case': True,
 'min_df': 2,
 'min_sentence_length': 3,
 'remove_infrequent': False,
 'remove_punctuation': False,
 'remove_stop_words': False,
 'replace_nums': True,
 'spelling_correct': True,
 'stem': False,
 'window_size': 9}

## Validate all essay sets are equal and the same

In [15]:
def names_the_same(essay_sets):
    unique_fnames = [] # list of sets of str (fnames)
    for essay_collection in essay_sets:
        names = set()
        for e in essay_collection:
            names.add(e.name)
        unique_fnames.append(names)
    for a in unique_fnames:
        print(len(a))
        for b in unique_fnames:
            assert len(a) == len(b), "lens don't match"
            assert a == b, "don't match"

In [16]:
names_the_same([ana_tagged_tr, untagged_essays_train])

870
870


In [17]:
names_the_same([ana_tagged_test, untagged_essays_test])

218
218


In [18]:
def essays_2_hash_map(essays):
    lu = {}
    for e in essays:
        lu[e.name] = e
    return lu

In [19]:
# checks the number of words and sentences are the same for 2 sets of essays
def validate_tagged_essays(essays_a, essays_b, tags_should_match=True):
    # make sure obj is not the same
    assert essays_a != essays_b
    print("Validating", len(essays_a), "essays")
    assert len(essays_a) == len(essays_b), "Lens don't match"
    
    a_hmap = essays_2_hash_map(essays_a)
    b_hmap = essays_2_hash_map(essays_b)
    
    # same essays?
    assert a_hmap.keys() == b_hmap.keys()
    intersect = set(a_hmap.keys()).intersection(b_hmap.keys())
    assert len(intersect) == len(a_hmap.keys())
    assert len(a_hmap.keys()) > 1    
    assert len(a_hmap.keys()) == len(b_hmap.keys())
    
    word_misses = 0
    
    for key, a_essay in a_hmap.items():
        b_essay = b_hmap[key]
        # assert NOT the same obj ref
        assert a_essay != b_essay
        assert len(a_essay.sentences) == len(b_essay.sentences)
        assert len(a_essay.sentences) > 0
        assert len(b_essay.sentences) > 0
        for i in range(len(a_essay.sentences)):
            a_sent = a_essay.sentences[i]
            b_sent = b_essay.sentences[i]
            # the same lists?
            #assert a_sent == b_sent
            assert len(a_sent) == len(b_sent)
            if not len(a_sent) == len(b_sent):
                print(key, "\tsent-ix:", i, "lens", len(a_sent), len(b_sent))
            for wd_ix, (a_wd, a_tags) in enumerate(a_sent):
                b_wd, b_tags = b_sent[wd_ix]
                if a_wd != b_wd:
                    word_misses+=1
                assert a_wd   == b_wd,   \
                    "Words don't match: '{a}' - '{b}', Esssay: {essay} Sent Ix: {i}".format(
                            a=a_wd, b=b_wd, essay=key, i=i)
                
                # SH - Make conditional, as untagged essays contain new anaphora tags
                if tags_should_match:                
                    assert a_tags == b_tags, \
                        "Tags don't match: '{a}' - '{b}', Esssay: {essay} Sent Ix: {i}".format(
                            a=str(a_tags), b=str(b_tags), essay=key, i=i)
                else:
                    intersectn = a_tags.intersection(b_tags)
                    # smaller set should match intersection i.e. be a subset of larger one
                    # will only differ due to new anaphora tags
                    if len(b_tags) <= len(a_tags):
                        assert intersectn == b_tags
                    else:
                        assert intersectn == a_tags
                        
    if word_misses > 0:
        print("Word miss-matches: ", word_misses)
    print("Validation Passed")
    return None

In [20]:
validate_tagged_essays(untagged_essays_train, ana_tagged_tr, tags_should_match=False)
validate_tagged_essays(untagged_essays_test, ana_tagged_test, tags_should_match=False)

Validating 870 essays
Validation Passed
Validating 218 essays
Validation Passed


## Re-Compute Accuracy Metrics

In [21]:
e = ana_tagged_tr[10]
len(e.sentences),len(e.pred_tagged_sentences)

(9, 9)

In [22]:
e = ana_tagged_test[10]
len(e.sentences),len(e.pred_tagged_sentences)

(11, 11)

In [23]:
assert len(ana_tagged_tr) > 2 * len(ana_tagged_test) # sanity check
len(ana_tagged_tr), len(ana_tagged_test)

(870, 218)

### Get Predicted and Actual Labels

In [24]:
def get_label(tag, expected_tag_set):
    if tag in expected_tag_set:
        return 1
    else:
        return 0

def get_wd_level_lbs(essays, expected_tags):
    expected_tags = set(expected_tags)
    ysbycode = defaultdict(list)
    for e in essays:
        for sent in e.sentences:
            for wd, tag_set in sent:
                for etag in expected_tags:
                    ysbycode[etag].append(get_label(etag, tag_set))
    return ysbycode    

# for pred tags
def get_wd_level_preds(essays, expected_tags):
    expected_tags = set(expected_tags)
    ysbycode = defaultdict(list)
    for e in essays:
        for sentix in range(len(e.sentences)):
            p_ccodes = e.pred_tagged_sentences[sentix]            
            for wordix in range(len(p_ccodes)):
                ptag_set = set([p_ccodes[wordix]])
                assert len(ptag_set) >=1, "No tags found"
                for exp_tag in expected_tags:
                    ysbycode[exp_tag].append(get_label(exp_tag, ptag_set))    
    return ysbycode   

## Validate Metrics Codes - Train CV

In [25]:
def metrics_to_df(metrics):
    import Rpfa

    rows = []
    for k,val in metrics.items():
        if type(val) == Rpfa.rpfa:
            d = dict(val.__dict__) # convert obj to dict
        elif type(val) == dict:
            d = dict(val)
        else:
            d = dict()
        d["code"] = k
        rows.append(d)
    return pd.DataFrame(rows)

def compute_metrics_from_essays(tagged_esssays, expected_tags):
    #TODO - get predicted ccodes and anaphora labels, merge into one set of preds and filter by expected_tags.
    # this give flexibility to look at anaphora, cc or both
    act_ys_bycode  = get_wd_level_lbs(  tagged_esssays, expected_tags)
    pred_ys_bycode = get_wd_level_preds(tagged_esssays, expected_tags)

    assert len(act_ys_bycode.keys()) == len(pred_ys_bycode.keys()) == len(expected_tags), "Miss-matched codes"
    first_tag = list(expected_tags)[0]
    last_tag  = list(expected_tags)[-1]
    assert len(act_ys_bycode[first_tag]) == len(pred_ys_bycode[first_tag]), "Different numbers of words"
    assert len(act_ys_bycode[last_tag])  == len(pred_ys_bycode[last_tag]), "Different numbers of words"

    #print(len(act_ys_bycode), len(act_ys_bycode[first_tag]), len(pred_ys_bycode), len(pred_ys_bycode[first_tag]))
    
    metrics = ResultsProcessor.compute_metrics(act_ys_bycode, pred_ys_bycode)
    mean_metrics = ResultsProcessor.compute_mean_metrics(act_ys_bycode, pred_ys_bycode)
    return metrics_to_df(mean_metrics)

## VD Metrics

In [29]:
tagged_esssays = ana_tagged_tr
expected_tags = ana_tags

#TODO - get predicted ccodes and anaphora labels, merge into one set of preds and filter by expected_tags.
# this give flexibility to look at anaphora, cc or both
act_ys_bycode  = get_wd_level_lbs(  tagged_esssays, expected_tags)
pred_ys_bycode = get_wd_level_preds(tagged_esssays, expected_tags)

assert len(act_ys_bycode.keys()) == len(pred_ys_bycode.keys()) == len(expected_tags), "Miss-matched codes"
first_tag = list(expected_tags)[0]
last_tag  = list(expected_tags)[-1]
assert len(act_ys_bycode[first_tag]) == len(pred_ys_bycode[first_tag]), "Different numbers of words"
assert len(act_ys_bycode[last_tag])  == len(pred_ys_bycode[last_tag]), "Different numbers of words"

#print(len(act_ys_bycode), len(act_ys_bycode[first_tag]), len(pred_ys_bycode), len(pred_ys_bycode[first_tag]))

metrics = ResultsProcessor.compute_metrics(act_ys_bycode, pred_ys_bycode)
mean_metrics = ResultsProcessor.compute_mean_metrics(act_ys_bycode, pred_ys_bycode)

In [33]:
pd.DataFrame([mean_metrics[ANAPHORA]])

,accuracy,data_points,f1_score,fn,fp,num_codes,precision,recall,tn,tp
0,0.996893,145471,0.350575,353.0,99.0,475,0.552036,0.256842,144897.0,122.0


In [26]:
METRICS_COLS = ["code","f1_score", "precision", "recall", "accuracy", "data_points"]

ana_tags = [ANAPHORA]
metrics_tr_df = compute_metrics_from_essays(ana_tagged_tr, ana_tags)
# filter to Anaphora and Micro F1
metrics_tr_df = metrics_tr_df[np.isin(metrics_tr_df["code"], [ANAPHORA, __MICRO_F1__])]
metrics_tr_df[METRICS_COLS]

,code,f1_score,precision,recall,accuracy,data_points
0,Anaphor,0.350575,0.552036,0.256842,0.996893,145471.0
5,MICRO_F1,0.350575,0.552036,0.256842,0.996893,145471.0


## Test Metrics

In [27]:
metrics_test_df = compute_metrics_from_essays(ana_tagged_test, ana_tags)
# filter to Anaphora and Micro F1
metrics_test_df = metrics_test_df[np.isin(metrics_test_df["code"], [ANAPHORA, __MICRO_F1__])]
metrics_test_df[METRICS_COLS]

,code,f1_score,precision,recall,accuracy,data_points
0,Anaphor,0.47343,0.505155,0.445455,0.996921,35402.0
5,MICRO_F1,0.47343,0.505155,0.445455,0.996921,35402.0


## TODO
- Pull code into common library (for computing metrics from predictions), and refactor to use it
- Move this logic into the Query folder (where i'd expect it to be), Update notes for Coref, referencing results for train and test (and where to get test data)
- Run on SC essays - update the optimal ANA tag essays, re-do essay merge step